# Objectives

• Create useful and beautiful visualizations

• Perform a sentimental analysis 

• Fit a regression model on reviews and score to see which words are more indicative to a higher/lower score





# Quick Observation from the raw csv file

•	On a certain day, a hotel receives multiple reviews (both positive and negative) from reviewers with different nationalities

•	Both Negative_Review and Positive Review columns contain review indicating that the review is neither negative nor negative. They are more like neutral reviews that can cause confusion when I train the models. These reviews are "No Negative", "No Positive"

•	There are null values in both columns of reviews


# Import Data 

In [22]:
# Batch import of all the neccessary libraries 

%matplotlib inline
import numpy as np #NumPy library provides a multidimensional array object, various derived objects, statistical operations, sorting
import pandas as pd #Pandas library offers data structures and operations for manipulating numerical tables and time series
import scipy as sp
import nltk

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
'''CountVectorizer, and TfidfVectorizer are what we will use 
to "convert text into a matrix of token counts"
'''
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer


In [23]:
# Import the csv file
path = r'C:\Users\Huong Pham\Documents\Graduate School\Winter 2019\Independent Study\\'
data = 'Hotel Reviews.csv'
df= pd.read_csv(path+data)


# Data Examination

In [24]:
print (df.dtypes)
print (df.shape)
print (df.columns)
print (df.describe())

Hotel_Address                                  object
Additional_Number_of_Scoring                    int64
Review_Date                                    object
Average_Score                                 float64
Hotel_Name                                     object
Reviewer_Nationality                           object
Negative_Review                                object
Review_Total_Negative_Word_Counts               int64
Total_Number_of_Reviews                         int64
Positive_Review                                object
Review_Total_Positive_Word_Counts               int64
Total_Number_of_Reviews_Reviewer_Has_Given      int64
Reviewer_Score                                float64
Tags                                           object
days_since_review                              object
Latitude                                      float64
Longtitude                                    float64
dtype: object
(515738, 17)
Index(['Hotel_Address', 'Additional_Number_of_Scoring',

In [25]:
display(df.describe(include=['O']))

,Hotel_Address,Review_Date,Hotel_Name,Reviewer_Nationality,Negative_Review,Positive_Review,Tags,days_since_review
count,515738,515738,515738,515738,515738,515738,515738,515738
unique,1493,731,1492,227,330011,412601,55242,731
top,163 Marsh Wall Docklands Tower Hamlets London ...,8/2/2017,Britannia International Hotel Canary Wharf,United Kingdom,No Negative,No Positive,"[' Leisure trip ', ' Couple ', ' Double Room '...",1 days
freq,4789,2585,4789,245246,127890,35946,5101,2585


In [26]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,Latitude,Longtitude
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968



Key Observations: 

    •	There are 17 columns and 551,738 records
    •	There are distinctive 1492 hotels being reviewed 
    •	227 unique nationalities of the reviewers
    •	The top reviews come from reviewers in United Kingdom 


# Data Preparation

I determine that the following attributes are not very helpful for my analysis: 

    •	Review_Total_Negative_Word_Counts: Total number of words in the negative review
    •	Review_Total_Positive_Word_Counts: Total number of words in the positive review
    •	Total_Number_of_Reviews_Revew_Has_Given
    •	Tags
    •	Review_Date
    •	Days_since_review
    •	Latitude
    •	Longtitude


In [27]:
# Create a new data frame that has all the desired attributes
# The new data frame is named "hotels"

hotels=df[['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review','Total_Number_of_Reviews', 'Positive_Review',
        'Reviewer_Score']].drop_duplicates().reset_index()

# Examine the new data frame 
print(hotels.shape)
hotels.head()

(515173, 11)


,index,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Total_Number_of_Reviews,Positive_Review,Reviewer_Score
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,1403,Only the park outside of the hotel was beauti...,2.9
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,1403,No real complaints the hotel was great great ...,7.5
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,1403,Location was good and staff were ok It is cut...,7.1
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,1403,Great location in nice surroundings the bar a...,3.8
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,1403,Amazing location and building Romantic setting,6.7


Since the data contains the full address of the hotel, I want to filter it by Country only. This will be helpful for my data visualization later.


In [28]:
# Create a new column 'Country' based on the 'Hotel_Address'

hotels['Country']=hotels['Hotel_Address'].apply(lambda x: x.split()[-1]).replace('Kingdom','UK') # truncate the end of the string and replace "Kingdom" by UK
hotels.Country.unique()

array(['Netherlands', 'UK', 'France', 'Spain', 'Italy', 'Austria'],
      dtype=object)

In [29]:
hotels.head() #The new column is at the end of the frame


,index,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Total_Number_of_Reviews,Positive_Review,Reviewer_Score,Country
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,1403,Only the park outside of the hotel was beauti...,2.9,Netherlands
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,1403,No real complaints the hotel was great great ...,7.5,Netherlands
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,1403,Location was good and staff were ok It is cut...,7.1,Netherlands
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,1403,Great location in nice surroundings the bar a...,3.8,Netherlands
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,1403,Amazing location and building Romantic setting,6.7,Netherlands


I decided to combine "Negative_Review" and "Positive_Review" into one column "Review"

In [30]:
hotels["Review"] = hotels["Negative_Review"] + hotels["Positive_Review"]


In [31]:
hotels.head() #The Review column is at the end

,index,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Total_Number_of_Reviews,Positive_Review,Reviewer_Score,Country,Review
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,1403,Only the park outside of the hotel was beauti...,2.9,Netherlands,I am so angry that i made this post available...
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,1403,No real complaints the hotel was great great ...,7.5,Netherlands,No Negative No real complaints the hotel was g...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,1403,Location was good and staff were ok It is cut...,7.1,Netherlands,Rooms are nice but for elderly a bit difficul...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,1403,Great location in nice surroundings the bar a...,3.8,Netherlands,My room was dirty and I was afraid to walk ba...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,1403,Amazing location and building Romantic setting,6.7,Netherlands,You When I booked with your company on line y...


In [32]:
# Since I already had the combined column, I will dropping unwanted columns 
new_list = hotels.drop(['Negative_Review', 'Positive_Review', 'Hotel_Address'], axis=1)

In [33]:
print(new_list.shape)

(515173, 10)


In [34]:
new_list.head()

,index,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Total_Number_of_Reviews,Reviewer_Score,Country,Review
0,0,194,8/3/2017,7.7,Hotel Arena,Russia,1403,2.9,Netherlands,I am so angry that i made this post available...
1,1,194,8/3/2017,7.7,Hotel Arena,Ireland,1403,7.5,Netherlands,No Negative No real complaints the hotel was g...
2,2,194,7/31/2017,7.7,Hotel Arena,Australia,1403,7.1,Netherlands,Rooms are nice but for elderly a bit difficul...
3,3,194,7/31/2017,7.7,Hotel Arena,United Kingdom,1403,3.8,Netherlands,My room was dirty and I was afraid to walk ba...
4,4,194,7/24/2017,7.7,Hotel Arena,New Zealand,1403,6.7,Netherlands,You When I booked with your company on line y...


In [35]:
new_list["Review"] = new_list["Review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

In [36]:
print(new_list.shape)

(515173, 10)


In [37]:
new_hotels = new_list.dropna(axis=0, subset=["Review"])

In [38]:
print(new_hotels.shape)

(515173, 10)


- As you can see from the output of the line above, the number of rows stays the same after dropping the null values. I tried different variations of codes that drop null values. None of the code works. 

- I decided to save the current dataframe "new_hotels" to a csv file in my local drive. I would delete the null values in Excel

In [39]:
import os #This module provides a portable way of using operating system dependent functionality. For example like writing and reading a file
path_d = r'C:\Users\Huong Pham\Documents\Graduate School\Winter 2019\\'
new_hotels.to_csv(os.path.join(path_d,'new_hotels.csv'))

- There were 236 null values in the "Review" column. They are now deleted from dataframe df2